#需要的包一股脑全放里

In [30]:
import urllib2, urllib,sys,os
import codecs,re,string,json
from subprocess import *
from collections import Counter

#分词标注部分

In [31]:
#分词与词性标注  以json格式输出
def divPos(input):
    uri_base = "http://ltpapi.voicecloud.cn/analysis/?"
    api_key  = "l3z7P327MjUbYOpUyR7m4kmobn7eVVrsaJlUSxlb" 
    text= urllib.quote(dataa)
    #语言云的真正调用方法    
    uri_base = "http://ltpapi.voicecloud.cn/analysis/?"       
    data = {
            "api_key"   : "l3z7P327MjUbYOpUyR7m4kmobn7eVVrsaJlUSxlb",
            "text"      : text,
            "format"    : "json",
            "has_key"   : "false",
            "pattern"   : "pos",               

            }
    params = urllib.urlencode(data)
    try:
        request  = urllib2.Request(uri_base)
        response = urllib2.urlopen(request, params)
        content  = response.read().strip()
        
        return content 
    except urllib2.HTTPError, e:
        print >> sys.stderr, e.reason


In [32]:
#json格式转化成plain格式
def jsonToPlain(contentJson):
        #只保留分词和标注输出
        #posResult=[word for word in sentence[0] for sentence in content for ]
        resultList=[]
        for paragraph in contentJson:
            sentenceList=[]
            if paragraph[0][-1][-1]== "wp":
                for word in paragraph[0][:-1]:
                    sentenceList.append(word[1]+'\t'+word[2])
            else:
                for word in paragraph[0]:
                    sentenceList.append(word[1]+'\t'+word[2])
            sentence='\n'.join(sentenceList).encode('utf-8')
            resultList.append(sentence+'\n'*2)
        
        return resultList

In [33]:
#打开文本文件，从中读出要 分词和词性标注的原材料
"""
f= open('nodupliteInput.txt', 'r')
dataa=f.read()
if dataa[:3] == codecs.BOM_UTF8:
    dataa = dataa[3:]
"""
#测试句子，直接输入
dataa=raw_input('输入命令：'.decode('utf-8')).decode('gb2312').encode('utf-8')#程序中字符串为utf-8，交互中键盘敲入为gb2312
#输入句子，调用分词标注函数，输出结果（json）
output=divPos(dataa)

#将分词标注结果输出到txt文档中
f1=open('json.txt','w')
f1.write(output)
f1.close()

输入命令：从五一广场出发过橘子洲大桥左转到四医院


In [34]:
contentJson=json.loads(output)

resultList=jsonToPlain(contentJson)
print resultList
f2=open('plain.txt','w')
f2.writelines(resultList)
f2.close()

['\xe4\xbb\x8e\tp\n\xe4\xba\x94\xe4\xb8\x80\tnt\n\xe5\xb9\xbf\xe5\x9c\xba\tn\n\xe5\x87\xba\xe5\x8f\x91\tv\n\xe8\xbf\x87\tu\n\xe6\xa9\x98\xe5\xad\x90\xe6\xb4\xb2\tns\n\xe5\xa4\xa7\xe6\xa1\xa5\tn\n\xe5\xb7\xa6\tnd\n\xe8\xbd\xac\tv\n\xe5\x88\xb0\tv\n\xe5\x9b\x9b\tm\n\xe5\x8c\xbb\xe9\x99\xa2\tn\n\n']


# crf语义标注部分

In [35]:
def train(template,trainTxt,model):      
    #returnCode = call('crf_learn.exe template crfTrain1.txt model1')
    call('crf_learn.exe '+template+' '+trainTxt+' '+ model)
def test(model,testTxt,testOutput):
    
    #Popen('crf_test.exe -m model1 crfTest1.txt >output1.txt', shell = True, stdout = PIPE).stdout
    Popen('crf_test.exe -m '+model+' '+ testTxt +'>'+testOutput, shell = True, stdout = PIPE).communicate()
def evaluate(testOutput,testResult):
    """
    #打开测试文件 并读入
    #f=open('output1.txt','r')
    #f=open(testOutput,'r')    
    #article=f.read()
    #f.close()
    #替换tab为space  能够让conlleval.pl识别的格式
    #article.replace(unicode('\t'),unicode(' '))
    #tabToSpace=re.compile(unicode("\t","utf-8"),re.S)
    #tabToSpace=re.compile("\t",re.S)
    #articleSpace=tabToSpace.sub(' ',article)
    #将评测结果输出到outputEvaluateResult.txt
    #f1=open('outputEvaluate1.txt','w')
    outputEvaluate='evaluate_'+testOutput
    f1=open(outputEvaluate,'w')
    f1.write(articleSpace)
    f1.close()
    """
    #evalue=Popen('conlleval.pl < outputEvaluate1.txt > outputEvaluateResult1.txt',
    #             shell = True, stdout = PIPE).stdout
    evalue=Popen('conlleval.pl -d "\t" -r < '+testOutput+' > '+testResult,
                 shell = True, stdout = PIPE).stdout
    #https://argcv.com/articles/2104.c#respond   参考资料，参数使用很全
    #print evalue.read()

In [36]:
train('template','crfTrain.txt','model')

In [37]:
test('model','plain.txt','output.txt')
#test('model','crfTest1.txt','output1.txt')

In [38]:
evaluate('output1.txt','output1Evaluate.txt')

#将名词和方向词合并

In [40]:
#将普通的字符串格式的输入转化成json列表形式
def plainToJson(crfOutput):
    fCrfTrain=open(crfOutput,'r')
    article=fCrfTrain.read()
    if article[:3] == codecs.BOM_UTF8:
            article = article[3:]  
    fCrfTrain.close()

    sentenceLabel=re.compile(r"(.*?)\n\n",re.S)
    sentenceList=re.findall(sentenceLabel,article)

    #print sentenceList[0:10]

    wordListInSentenceList=[]
    for sentence in sentenceList:
        wordList=sentence.split('\n')
        wordPosYuyiList=[]
        for word in wordList:
            wordPosYuyi=word.split('\t')
            wordPosYuyiList.append(wordPosYuyi)
        wordListInSentenceList.append(wordPosYuyiList)
    return wordListInSentenceList

In [41]:
#将名词和方向词合并 输出json格式
def n_f(wordListInSentenceList):
    for s in xrange(len(wordListInSentenceList)):
        wordPosition=0
        while wordPosition<len(wordListInSentenceList[s]):
            nPosition=[]
            while wordPosition<len(wordListInSentenceList[s]) and '7_' in wordListInSentenceList[s][wordPosition][-1]:
                
                nPosition.append(wordPosition)
                wordPosition+=1
            #print nPosition 
            if len(nPosition)!=0:
                #
                words=[]
                for p in nPosition:
                    words.append(wordListInSentenceList[s][p][0])
                word=''.join(words)
                nTotal=[word,'ns','7']
                wordListInSentenceList[s]=wordListInSentenceList[s][:nPosition[0]]+[nTotal]+wordListInSentenceList[s][nPosition[-1]+1:]
                #print wordListInSentenceList[s]
                wordPosition=nPosition[0]
                
            fPosition=[]
            while wordPosition<len(wordListInSentenceList[s]) and '2_' in wordListInSentenceList[s][wordPosition][-1]:
                
                fPosition.append(wordPosition)
                wordPosition+=1
            #print nPosition 
            if len(fPosition)!=0:
                #
                words=[]
                for p in fPosition:
                    words.append(wordListInSentenceList[s][p][0])
                word=''.join(words)
                fTotal=[word,'nd','2']
                wordListInSentenceList[s]=wordListInSentenceList[s][:fPosition[0]]+[fTotal]+wordListInSentenceList[s][fPosition[-1]+1:]
                #print wordListInSentenceList[s]
                wordPosition=fPosition[0]
            #print wordPosition
            wordPosition+=1
    return

In [42]:
#将合并后的json格式转化成plain格式输出    
def jsonToPlain1(contentJson):
        #只保留分词和标注输出
        #posResult=[word for word in sentence[0] for sentence in content for ]
        resultList=[]
        for paragraph in contentJson:
            sentenceList=[]
            for word in paragraph:
                sentenceList.append(word[0]+'\t'+word[1]+'\t'+word[2])
            sentence='\n'.join(sentenceList)
            resultList.append(sentence+'\n'*2)
        return resultList

In [44]:
wordListInSentenceList=plainToJson('output.txt')#crfTrain.txt  包含词  词性  语义 的plain文件
n_f(wordListInSentenceList)

data=json.dumps(wordListInSentenceList)
f3=open('n_f_outputTestJson.txt','w')                #jsonData.txt对应crfTrain的合并结果
f3.writelines(data)
f3.close()

result=jsonToPlain1(wordListInSentenceList)
f4=open('n_f_outputTest.txt','w')           #n_f.data对应jsonData.txt  plain格式
f4.writelines(result)
f4.close()

#svm判断是否

In [51]:
#所有句子的特征列表
def sentProperty(listData):
    property_all=[]
    for sentence in listData:
        #句子中的语义列表
        yuyiList=[ word[-1] for word in sentence]
        yuyiStr=' '+' '.join(yuyiList)
        #除0外的词占句子的比例
        property1=1-Counter(yuyiList)['0']/len(yuyiList)
        #是否有动词
        property2=1 if re.search(r'3_|4',yuyiStr) else 0
        
        #是否含有方向词
        property3=1 if '2' in yuyiList else 0
        #是否含有地名词
        property4=1 if '7' in yuyiList else 0
        #是否含有数词加量词的集合
        property5=1 if  re.search(r'6 8',yuyiStr)  else 0
        #方向词加动词的个数
        property6=len(re.findall(r'2 [34]',yuyiStr))
        #动词加地名词的个数
        property7=len(re.findall(r'3_? 7| 4 7',yuyiStr))
        #地名词加动词
        property8=len(re.findall(r'7 [34]',yuyiStr))
        #动词的个数
        property9=len(re.findall(r'3_| 4',yuyiStr))
        #关键词类别个数占句子长度的比例
        property10=len(set(yuyiList))/len(yuyiList)
        
        property_all.append([property1,property2,property3,property4,property5,property6,property7,property8,property9,property10])
    return property_all

In [52]:
#给每个句子贴上1或-1的标签，自己标出来
def label_initialize():
    set_label_1=[1]*60
    label_0=[7,53,54,55,56,57,58,59]
    for label in label_0:
        set_label_1[label]=-1
    return set_label_1

In [53]:
#将句子的特征和1、-1标签弄成svm的输入格式
def svm_input(property_all,label_tag):
    svm_format=[]
    for index in xrange(len(property_all)):               
        column=str(label_tag[index])
        for property_index in xrange(len(property_all[index])):
            column+='\t'+str(property_index+1)+':'+str(property_all[index][property_index])   
        svm_format.append(column+'\n')
    return svm_format

In [58]:
#从合并后的带有语义标注的预料中  提取出svm的特征向量  以及自己标注的标签 存放在一个文本文档中
f5=open('n_f_outputTestJson.txt','r')  #jsonData
jsonData1=f5.read()
f5.close()
listData=json.loads(jsonData1)

sentPropertyAll=sentProperty(listData)

#sentLabel=label_initialize()
sentLabel=[0]*len(sentPropertyAll)
svm_format=svm_input(sentPropertyAll,sentLabel)

f6=open('svm_input_test.txt','w')
f6.writelines(svm_format)
f6.close()

In [69]:
#训练模型  并进行测试
from svmutil import *
y,x=svm_read_problem('svm_input_test.txt')
#m=svm_train(sentLabel,sentPropertyAll,'-c 4')
#svm_save_model('model1', m)
m = svm_load_model('model1')

p_label,p_acc,p_val=svm_predict(y,x,m)
print  p_label,p_acc,p_val

Accuracy = 100% (1/1) (classification)
[0.0] (100.0, 0.0, nan) [[1]]


#crf分句

In [72]:
train('templateSplit','crfSentSplitTrain.txt','modelSplit')

In [73]:
test('modelSplit','n_f_outputTest.txt','outputSplit.txt')

#crf起止点识别

In [75]:
train('templateStartFinish','crfStartFinishTrain.txt','modelStartFinish')

In [76]:
test('modelStartFinish','outputSplit.txt','outputStartFinish.txt')